<font color="green"><h2> **Welcome to ABT/HYD 182**


 ## **Lab 9**: GeoAI – Instance Segmentation (Mask R-CNN)


Fill in the blanks and follow the hints in each exercise.

### **Due Date: March 11 | 11:59 PM | 2026**
------------------------------------------------------------------------------

## Academic Integrity Statement

**This work was completed without the use of Generative AI tools (such as ChatGPT, Copilot, etc.).**

By completing the information below, you certify that you have completed this assignment independently and without the assistance of generative AI tools. This lab is designed to help you learn GeoAI and instance segmentation through hands-on practice.

---

**Note:** If you did use a generative AI tool, you must clearly disclose this in your notebook, including which tool you used and how you used it (e.g., debugging, understanding error messages, or clarifying concepts). Failure to disclose the use of generative AI tools may be considered a violation of course academic integrity policies.

In [ ]:
# Enter your information below
your_name = ""  # Replace with your full name
date = ""       # Replace with today's date (e.g., "March 5, 2026")

# Print your statement
print("Academic Integrity Statement")
print("=" * 50)
print(f"Name: {your_name}")
print(f"Date: {date}")
print("=" * 50)
print("I certify that this work was completed without the use of Generative AI tools.")

## **Table of Contents**

1. [Exercise 1](#section1) – Install packages & import libraries  
2. [Exercise 2](#section2) – Download sample data  
3. [Exercise 3](#section3) – Visualize sample data  
4. [Exercise 4](#section4) – Create training data  
5. [Exercise 5](#section5) – Train instance segmentation model  
6. [Exercise 6](#section6) – Run inference  
7. [Exercise 7](#section7) – Vectorize masks & add geometric properties  
8. [Exercise 8](#section8) – Visualize results  
9. [Exercise 9](#section9) – Filter by area & compare predictions  
10. [Exercise 10](#section10) – Model performance  
11. [Exercise 11 (Bonus)](#section_bonus11) – Setup for bonus (packages & directories)  
12. [Exercise 12 (Bonus)](#section_bonus12) – Download solar panel data  
13. [Exercise 13 (Bonus)](#section_bonus13) – Visualize solar panel imagery  
14. [Exercise 14 (Bonus)](#section_bonus14) – Run solar panel detection  
15. [Exercise 15 (Bonus)](#section_bonus15) – Vectorize masks & add geometric properties  
16. [Exercise 16 (Bonus)](#section_bonus16) – Filter and visualize results  

--------------------------------------------
Learning objectives
---------------------------------------------

* In this lab you will:

    *   install and use the **geoai** package for geospatial AI
    *   download sample imagery and vector labels for building detection
    *   create training tiles and train a **Mask R-CNN** instance segmentation model
    *   run inference, vectorize masks, and add geometric properties
    *   visualize and compare predictions with imagery
    *   interpret training metrics and understand instance vs semantic segmentation

This notebook is based on training instance segmentation models for object detection (e.g., building detection) using Mask R-CNN. Unlike semantic segmentation, instance segmentation distinguishes between individual objects of the same class.

Resources:
- [GeoAI documentation](https://geoai.readthedocs.io/)
- [OpenGeos GeoAI examples](https://github.com/opengeos/geoai)

<a name="section1"></a>
## **Exercise 1** – Install packages & import libraries

**What you'll do:** Install the GeoAI package (uncomment the pip line if needed), then import it.

**Hint:** Run the GPU check cell first. Then uncomment `# %pip install geoai-py` if the package is not installed, and run `import geoai`.

### Check GPU (required)

This lab runs on GPU. **Colab cannot enable GPU from code**—you must choose it once in the menu:

1. Click **Runtime** (or the **▶ Connect** dropdown) → **Change runtime type**  
2. Set **Hardware accelerator** to **T4 GPU** → **Save**  
3. Re-run the notebook from the top  

After that, the check below will report "GPU OK".

In [ ]:
# Check GPU (Colab cannot enable GPU from code—you must set it in the menu once)
try:
    gpu_name = __import__("subprocess").check_output(
        ["nvidia-smi", "--query-gpu=name", "--format=csv,noheader"], text=True
    ).strip().split("\n")[0]
except Exception:
    gpu_name = ""
if not gpu_name or "T4" not in gpu_name.upper():
    print("WARNING: T4 GPU not detected.")
    print("  → Runtime → Change runtime type → Hardware accelerator: T4 GPU → Save")
    print("  → Then re-run the notebook from the top.")
else:
    print("GPU OK:", gpu_name)

In [ ]:
%pip install geoai-py

## Import libraries

### Colab: Pro or free?

Run the next cell and answer the prompt: **Are you using Google Colab Pro? (True/False)**. Type `True` or `False` and press Enter. Settings will adjust automatically: Pro uses larger batches and training visuals; free uses batch size 1 and fewer windows to avoid out-of-memory errors.

In [ ]:
# Answer the prompt: Are you using Colab Pro? (True/False)
reply = input("Are you using Google Colab Pro? (True/False): ").strip().lower()
COLAB_PRO = reply in ("true", "t", "yes", "y", "1")

USE_LOW_RAM = not COLAB_PRO

if USE_LOW_RAM:
    TILE_SIZE = 256
    STRIDE = 256
    BATCH_SIZE = 1
    WINDOW_SIZE = 256
    OVERLAP = 64
    NUM_EPOCHS = 5
    VISUALIZE_TRAINING = False
else:
    TILE_SIZE = 512
    STRIDE = 256
    BATCH_SIZE = 4
    WINDOW_SIZE = 512
    OVERLAP = 256
    NUM_EPOCHS = 10
    VISUALIZE_TRAINING = True
print("COLAB_PRO =", COLAB_PRO, "| USE_LOW_RAM =", USE_LOW_RAM)

In [ ]:
import geoai

### Mount Google Drive and create lab folder

All data, tiles, models, and outputs will be saved under **My Drive → ABT182_GeoAI** so you can reuse them and avoid re-downloading.

In [ ]:
from google.colab import drive
import os

drive.mount("/content/drive")

# Base folder in your Google Drive
BASE_DIR = "/content/drive/MyDrive/ABT182_GeoAI"
os.makedirs(f"{BASE_DIR}/data/train", exist_ok=True)
os.makedirs(f"{BASE_DIR}/data/test", exist_ok=True)
os.makedirs(f"{BASE_DIR}/tiles", exist_ok=True)
os.makedirs(f"{BASE_DIR}/models", exist_ok=True)
os.makedirs(f"{BASE_DIR}/outputs", exist_ok=True)
print(f"Created: {BASE_DIR}")
print("  data/train, data/test, tiles, models, outputs")

<a name="section2"></a>
## **Exercise 2** – Download sample data

**What this section does:** Download the training raster, vector, and test raster from the URLs, then copy them to your Drive so paths are under `BASE_DIR`. Paths are given below so you don't have to guess.

**Fill in (small):** In the download/copy cell, replace only the variable names: `FILL_TRAIN_URL` → `train_raster_url`, `FILL_VECTOR_URL` → `train_vector_url`, `FILL_TEST_URL` → `test_raster_url`; then in `shutil.copy` use `_train`, `train_raster_path` (and similarly for vector and test).

In [ ]:
train_raster_url = (
    "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/naip_rgb_train.tif"
)
train_vector_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/naip_train_buildings.geojson"
test_raster_url = (
    "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/naip_test.tif"
)

In [ ]:
import shutil

# Paths (given so you are not confused). FILL only the variable names in the lines below.
_train = geoai.download_file(FILL_TRAIN_URL)   # hint: train_raster_url
_vector = geoai.download_file(FILL_VECTOR_URL) # hint: train_vector_url
_test = geoai.download_file(FILL_TEST_URL)     # hint: test_raster_url

train_raster_path = f"{BASE_DIR}/data/train/naip_rgb_train.tif"
train_vector_path = f"{BASE_DIR}/data/train/naip_train_buildings.geojson"
test_raster_path = f"{BASE_DIR}/data/test/naip_test.tif"

shutil.copy(FILL_SRC1, FILL_DST1)  # hint: _train, train_raster_path
shutil.copy(FILL_SRC2, FILL_DST2)  # hint: _vector, train_vector_path
shutil.copy(FILL_SRC3, FILL_DST3)  # hint: _test, test_raster_path
print("Saved to Drive:", BASE_DIR)

<a name="section3"></a>
## **Exercise 3** – Visualize sample data

**What this section does:** View training raster metadata, then map NAIP with building footprints (raster first, then vectors). Paths are given.

**Fill in (small):** In the first map cell, replace `FILL_BANDS` with the band list for RGB, e.g. `[1, 2, 3]`. Do the same in the test image cell.

In [ ]:
geoai.get_raster_info(train_raster_path)

In [ ]:
# Leafmap: add NAIP first, then vectors. FILL: bands for RGB, e.g. [1, 2, 3]
import leafmap
m = leafmap.Map()
m.add_raster(train_raster_path, layer_name="NAIP", bands=FILL_BANDS)
m.add_geojson(train_vector_path, layer_name="Buildings", style={"stroke": True, "color": "#ff0000", "weight": 2, "fill": False, "fillOpacity": 0})
m

In [ ]:
# Test NAIP image. FILL: same bands for RGB, e.g. [1, 2, 3]
import leafmap
m = leafmap.Map()
m.add_raster(test_raster_path, layer_name="Test NAIP", bands=FILL_BANDS)
m

<a name="section4"></a>
## **Exercise 4** – Create training data

**What this section does:** Create training tiles from the raster and vector; `out_folder` path is given.

**Fill in (small):** Replace only: `FILL_RASTER` → `train_raster_path`, `FILL_VECTOR` → `train_vector_path`, `FILL_TILE_SIZE` → `TILE_SIZE`, `FILL_STRIDE` → `STRIDE`.

In [ ]:
out_folder = f"{BASE_DIR}/tiles/buildings_instance"
tiles = geoai.export_geotiff_tiles(
    in_raster=FILL_RASTER,      # hint: train_raster_path
    out_folder=out_folder,
    in_class_data=FILL_VECTOR,  # hint: train_vector_path
    tile_size=FILL_TILE_SIZE,   # hint: TILE_SIZE
    stride=FILL_STRIDE,         # hint: STRIDE
    buffer_radius=0,
)

<a name="section5"></a>
## **Exercise 5** – Train instance segmentation model

**What this section does:** Train Mask R-CNN on the tiles; paths under `out_folder` are used.

**Fill in (small):** Replace only: `FILL_IMAGES_DIR` → `f"{out_folder}/images"`, `FILL_LABELS_DIR` → `f"{out_folder}/labels"`, `FILL_OUTPUT_DIR` → `f"{out_folder}/instance_models"`.

In [ ]:
geoai.train_instance_segmentation_model(
    images_dir=FILL_IMAGES_DIR,   # hint: f"{out_folder}/images"
    labels_dir=FILL_LABELS_DIR,   # hint: f"{out_folder}/labels"
    output_dir=FILL_OUTPUT_DIR,   # hint: f"{out_folder}/instance_models"
    num_classes=2,
    num_channels=3,
    batch_size=BATCH_SIZE,
    num_epochs=NUM_EPOCHS,
    learning_rate=0.005,
    val_split=0.2,
    visualize=VISUALIZE_TRAINING,
    verbose=True,
)

<a name="section6"></a>
## **Exercise 6** – Run inference

**What this section does:** Run the trained model on the test image; output mask path is given so you are not confused.

**Fill in (small):** In the inference call only, replace `FILL_INPUT` → `test_raster_path`, `FILL_OUTPUT` → `masks_path`, `FILL_MODEL` → `model_path`.

In [ ]:
# Paths given (no fill needed)
masks_path = f"{BASE_DIR}/outputs/naip_test_instance_prediction.tif"
model_path = f"{out_folder}/instance_models/best_model.pth"

In [ ]:
# FILL: input_path=FILL_INPUT (test_raster_path), output_path=FILL_OUTPUT (masks_path), model_path=FILL_MODEL (model_path)
import gc
geoai.instance_segmentation(
    input_path=FILL_INPUT,
    output_path=FILL_OUTPUT,
    model_path=FILL_MODEL,
    num_classes=2,
    num_channels=3,
    window_size=WINDOW_SIZE,
    overlap=OVERLAP,
    confidence_threshold=0.5,
    batch_size=BATCH_SIZE,
)
gc.collect()

<a name="section7"></a>
## **Exercise 7** – Vectorize masks & add geometric properties

**What this section does:** Convert the mask raster to polygons and add area (m²), perimeter, etc. Path for output GeoJSON is given.

**Fill in (small):** In the first cell, replace `FILL_MASK_PATH` with the variable holding the mask raster path (e.g. `masks_path`). In the second, replace `FILL_GDF` with the GeoDataFrame from the previous step (e.g. `gdf`).

In [ ]:
output_vector_path = f"{BASE_DIR}/outputs/naip_test_instance_prediction.geojson"
gdf = geoai.orthogonalize(FILL_MASK_PATH, output_vector_path, epsilon=2)  # hint: masks_path

In [ ]:
gdf_props = geoai.add_geometric_properties(FILL_GDF, area_unit="m2", length_unit="m")  # hint: gdf

<a name="section8"></a>
## **Exercise 8** – Visualize results

**What this section does:** Show popup text in black (run the CSS cell first), then NAIP + predicted masks, a split map (NAIP only vs masks only), and buildings colored by area. Paths are given.

**Fill in (small):** In the map cells, replace: `FILL_BANDS` → e.g. `[1, 2, 3]` for RGB; `FILL_LAYER_NAME` → e.g. `"Predicted masks"`; `FILL_CMAP` → e.g. `"YlOrRd"` for area colors; `FILL_SCHEME` → e.g. `"Quantiles"`.

In [ ]:
# Run this first so popup/tooltip text is black (readable on white background)
from IPython.display import display, HTML
display(HTML("""
<style>
.leaflet-popup-content-wrapper, .leaflet-popup-content { color: #000 !important; }
.leaflet-tooltip { color: #000 !important; }
</style>
"""))

In [ ]:
# NAIP + masks. FILL: bands (e.g. [1,2,3]), layer_name (e.g. "Predicted masks")
import leafmap
m = leafmap.Map()
m.add_raster(test_raster_path, layer_name="NAIP", bands=FILL_BANDS)
m.add_raster(masks_path, layer_name=FILL_LAYER_NAME, cmap="tab20", nodata=0)
m

In [ ]:
# Compare with/without masks: two maps side by side (split_map does not work with local rasters)
import leafmap
from ipywidgets import HBox
from IPython.display import display
m_left = leafmap.Map()
m_left.add_raster(test_raster_path, layer_name="NAIP only", bands=FILL_BANDS)
m_right = leafmap.Map()
m_right.add_raster(test_raster_path, layer_name="NAIP", bands=FILL_BANDS)
m_right.add_raster(masks_path, layer_name=FILL_LAYER_NAME, cmap="tab20", nodata=0)
display(HBox([m_left, m_right]))

In [ ]:
# Buildings colored by area. FILL: bands, scheme (e.g. "Quantiles"), cmap (e.g. "YlOrRd")
import leafmap
m = leafmap.Map()
m.add_raster(test_raster_path, layer_name="NAIP", bands=FILL_BANDS)
m.add_data(gdf_props, column="area_m2", scheme=FILL_SCHEME, cmap=FILL_CMAP, legend_title="Area (m²)")
m

After Exercise 8 – **Histogram:** Make a simple histogram of `area_m2` to see the distribution of building sizes. **Fill in (small):** e.g. choose `bins=25` or another number.

In [ ]:
# FILL: e.g. bins=25. What this does: histogram of building areas (area_m2)
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 4))
plt.hist(gdf_props["area_m2"], bins=FILL_BINS, color="steelblue", edgecolor="white")
plt.xlabel("Area (m²)"); plt.ylabel("Count"); plt.title("Distribution of predicted building areas")
plt.tight_layout(); plt.show()

<a name="section9"></a>
## **Exercise 9** – Filter by area & compare predictions

**What this section does:** Keep only buildings above a minimum area (e.g. 50 m²), then map filtered buildings and compare with imagery.

**Fill in (small):** Replace `FILL_MIN_AREA` with a number in m² (e.g. `50`) to filter out small detections.

In [ ]:
gdf_filtered = gdf_props[(gdf_props["area_m2"] > FILL_MIN_AREA)]  # hint: e.g. 50

In [ ]:
# NAIP + filtered buildings (paths and params given)
import leafmap
m = leafmap.Map()
m.add_raster(test_raster_path, layer_name="NAIP", bands=[1, 2, 3])
m.add_data(gdf_filtered, column="area_m2", scheme="Quantiles", cmap="YlOrRd", legend_title="Area (m²)")
m

## Compare predictions with imagery

In [ ]:
# Compare view (colored by area_m2)
import leafmap
m = leafmap.Map()
m.add_raster(test_raster_path, layer_name="NAIP", bands=[1, 2, 3])
m.add_data(gdf_filtered, column="area_m2", scheme="Quantiles", cmap="RdYlBu_r", legend_title="Area (m²)")
m

**Box plot by size range:** Group buildings into size ranges and plot a box plot. **Fill in (small):** e.g. define bins and labels for `pd.cut()` (e.g. bins=[0, 100, 250, 500, 2000], labels=["0-100", "100-250", "250-500", "500+"]).

In [ ]:
# FILL: bins and labels for size_range. Boxes use filled colors (Blues colormap).
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
gdf_f = gdf_filtered.copy()
gdf_f["size_range"] = pd.cut(gdf_f["area_m2"], bins=FILL_BINS, labels=FILL_LABELS)
fig, ax = plt.subplots(figsize=(8, 4))
labels = FILL_LABELS  # same order as in pd.cut above
data = [gdf_f[gdf_f["size_range"] == lb]["area_m2"].values for lb in labels]
bp = ax.boxplot(data, labels=labels, patch_artist=True)
colors = plt.cm.Blues(np.linspace(0.35, 0.85, len(labels)))
for patch, color in zip(bp["boxes"], colors):
    patch.set_facecolor(color)
ax.set_xlabel("Area range (m²)"); ax.set_ylabel("Area (m²)"); ax.set_title("Building areas by size range")
plt.tight_layout(); plt.show()

<a name="section10"></a>
## **Exercise 10** – Model performance

**What this section does:** Plot training/validation loss and accuracy from the saved history. Path is given below; just run the cell.

In [ ]:
geoai.plot_performance_metrics(
    history_path=f"{out_folder}/instance_models/training_history.pth",
    figsize=(15, 5),
    verbose=True,
)

---
## **Bonus: Solar Panel Detection**

The following exercises are **optional**. They use a **pre-trained** solar panel detector (no training step), so they are lighter on RAM and suitable for Colab free tier or for doing later. If you run only the bonus section, run **Exercise 11 (Bonus)** first to set up packages and directories; otherwise you can skip it.

<a name="section_bonus11"></a>
## **Exercise 11 (Bonus)** – Setup for bonus (packages & directories)

**What you'll do:** If you are running only the bonus section, run this cell to mount Drive, set `BASE_DIR`, create folders, and set Colab Pro if needed. If you already ran the full notebook from the top, skip this cell. If `geoai` is not installed, run `%pip install geoai-py` in a cell above first. No fill required.

In [ ]:
# Run this only if you did NOT run from the top of the notebook. If geoai not installed: %pip install geoai-py
from google.colab import drive
import os

drive.mount("/content/drive")

BASE_DIR = "/content/drive/MyDrive/ABT182_GeoAI"
os.makedirs(f"{BASE_DIR}/data/solar", exist_ok=True)
os.makedirs(f"{BASE_DIR}/outputs", exist_ok=True)

try:
    _ = COLAB_PRO
except NameError:
    reply = input("Are you using Google Colab Pro? (True/False): ").strip().lower()
    COLAB_PRO = reply in ("true", "t", "yes", "y", "1")
    USE_LOW_RAM = not COLAB_PRO

import geoai
print("BASE_DIR =", BASE_DIR)

<a name="section_bonus12"></a>
## **Exercise 12 (Bonus)** – Download solar panel data

**What you'll do:** Download the solar panel sample raster and save it to Drive. **Fill in:** Replace `FILL_URL` with the variable holding the Hugging Face URL (e.g. `solar_raster_url`); in `shutil.copy` use `FILL_SRC` for the downloaded file (e.g. `_downloaded`) and `FILL_DST` for the destination path (e.g. `solar_raster_path`).

In [ ]:
import shutil

solar_raster_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/solar_panels_davis_ca.tif"
solar_raster_path = f"{BASE_DIR}/data/solar/solar_panels_davis_ca.tif"
_downloaded = geoai.download_file(FILL_URL)   # hint: solar_raster_url
shutil.copy(FILL_SRC, FILL_DST)               # hint: _downloaded, solar_raster_path
print("Saved:", solar_raster_path)

<a name="section_bonus13"></a>
## **Exercise 13 (Bonus)** – Visualize solar panel imagery

**What you'll do:** Inspect raster metadata and view the image in leafmap. **Fill in:** Use the raster path variable (e.g. `solar_raster_path`) for `get_raster_info` and `add_raster`; use RGB bands e.g. `[1, 2, 3]`.

In [ ]:
geoai.get_raster_info(FILL_RASTER_PATH)  # hint: solar_raster_path

In [ ]:
import leafmap
m = leafmap.Map()
m.add_raster(FILL_RASTER_PATH, layer_name="NAIP (Davis, CA)", bands=FILL_BANDS)  # hint: solar_raster_path, [1,2,3]
m

<a name="section_bonus14"></a>
## **Exercise 14 (Bonus)** – Run solar panel detection

**What you'll do:** Use the pre-trained `SolarPanelDetector` to generate masks. **Fill in:** In `generate_masks`, use `FILL_INPUT` for the raster path (e.g. `solar_raster_path`) and `FILL_OUTPUT` for the output path (e.g. `solar_masks_path`). Define `solar_masks_path` above if needed (e.g. `f\"{BASE_DIR}/outputs/solar_panel_masks.tif\"`).

In [ ]:
solar_masks_path = f"{BASE_DIR}/outputs/solar_panel_masks.tif"
detector = geoai.SolarPanelDetector()
solar_masks_path = detector.generate_masks(
    FILL_INPUT,   # hint: solar_raster_path
    output_path=FILL_OUTPUT,  # hint: solar_masks_path
    confidence_threshold=0.4,
    mask_threshold=0.5,
    min_object_area=100,
    overlap=0.25,
    chip_size=(400, 400),
    batch_size=4,
    verbose=False,
)
print("Masks saved:", solar_masks_path)

In [ ]:
# View NAIP + masks. FILL: raster path and masks path (e.g. solar_raster_path, solar_masks_path)
# Mask raster is single-band; use indexes=[1] so leafmap does not request bands 2–3
import leafmap
m = leafmap.Map()
m.add_raster(FILL_RASTER_PATH, layer_name="NAIP", bands=[1, 2, 3])
m.add_raster(FILL_MASKS_PATH, layer_name="Solar panel masks", indexes=[1], colormap="autumn", nodata=0)
m

<a name="section_bonus15"></a>
## **Exercise 15 (Bonus)** – Vectorize masks & add geometric properties

**What you'll do:** Convert the mask raster to polygons and add area (m²), etc. **Fill in:** In `orthogonalize`, use `FILL_MASK_PATH` (e.g. `solar_masks_path`) and `FILL_VECTOR_PATH` (e.g. `solar_vector_path`). In `add_geometric_properties`, use the GeoDataFrame from the previous step (e.g. `gdf_solar`).

In [ ]:
solar_vector_path = f"{BASE_DIR}/outputs/solar_panel_masks.geojson"
gdf_solar = geoai.orthogonalize(FILL_MASK_PATH, FILL_VECTOR_PATH, epsilon=0.2)  # hint: solar_masks_path, solar_vector_path
gdf_solar = geoai.add_geometric_properties(FILL_GDF, area_unit="m2", length_unit="m")  # hint: gdf_solar
gdf_solar.head()

<a name="section_bonus16"></a>
## **Exercise 16 (Bonus)** – Filter and visualize results

**What you'll do:** Filter by elongation and minimum area, then map polygons colored by area and show area distribution. **Fill in:** Choose numeric thresholds for `elongation` and `area_m2` (e.g. `< 10` and `> 5`). In `add_data`, use the column name for area (e.g. `"area_m2"`), and scheme/cmap (e.g. `"Quantiles"`, `"YlOrRd"`).

In [ ]:
# FILL: thresholds for elongation and area_m2 (e.g. 10 and 5)
gdf_solar_filtered = gdf_solar[(gdf_solar["elongation"] < FILL_ELONG) & (gdf_solar["area_m2"] > FILL_MIN_AREA)]
print("Filtered count:", len(gdf_solar_filtered))

In [ ]:
# FILL: column (e.g. "area_m2"), scheme (e.g. "Quantiles"), cmap (e.g. "YlOrRd")
import leafmap
m = leafmap.Map()
m.add_raster(solar_raster_path, layer_name="NAIP", bands=[1, 2, 3])
m.add_data(gdf_solar_filtered, column=FILL_COL, scheme=FILL_SCHEME, cmap=FILL_CMAP, legend_title="Area (m²)")
m

In [ ]:
# Histogram of solar panel areas; total area
import matplotlib.pyplot as plt
gdf_solar_filtered["area_m2"].hist(bins=FILL_BINS, color="steelblue", edgecolor="white")  # hint: e.g. 25
plt.xlabel("Area (m²)"); plt.ylabel("Count"); plt.title("Distribution of solar panel areas")
plt.tight_layout(); plt.show()
print("Total area (m²):", gdf_solar_filtered["area_m2"].sum())

In [ ]:
# Save filtered polygons. Path is given; just run the cell.
gdf_solar_filtered.to_file(f"{BASE_DIR}/outputs/solar_panels_filtered.geojson", driver="GeoJSON")
print("Saved:", f"{BASE_DIR}/outputs/solar_panels_filtered.geojson")

---
## **Credits**

This lab uses the **[GeoAI](https://opengeoai.org/)** Python package. We thank **Dr. Qiusheng Wu** for creating GeoAI and for the examples that inspired this tutorial.

For more information, documentation, and examples, visit: **https://opengeoai.org/**

<a name="section_submit"></a>
## **How to Submit Lab 9**

Once you have finished the exercises, follow the steps below to submit your assignment.

### Step 1: Run the Entire Notebook
- Run all code cells to make sure your notebook works correctly and displays all results.
- Go to the **Runtime** tab and click **Run all**.

### Step 2: Check Your Work
- Make sure you followed all instructions.
- Confirm that you completed the Academic Integrity cell and that all outputs look correct.

### Step 3: Rename and Save the Notebook
- Click on the notebook name at the **top left** of the page.
- Rename the file by replacing the default name with your own (e.g., **lastname_firstname_lab9.ipynb**).
- Save the notebook after renaming.

### Step 4: Create PDF Version
- Creating a PDF version of your notebook is **required**.
- **File** → **Print** → set **Destination** to **Save as PDF**, or **File** → **Download** → **Download .pdf**.

### Step 5: Submit Both Files to Canvas
- **Upload BOTH files** to the **Canvas** assignment for Lab 9:
  1. The renamed notebook (`.ipynb`) – **REQUIRED**
  2. The PDF (`.pdf`) – **REQUIRED**
- Submit before **March 11, 11:59 PM 2026**.

### Use of Generative AI Tools (GenAI Policy)
- This course **discourages the use of generative AI tools** (such as ChatGPT) for completing assignments, so you can develop your own skills.
- If you **do use a generative AI tool**, you must **clearly disclose** it in your notebook (which tool and how you used it).
- Failure to disclose may be considered a violation of course academic integrity policies.